In [0]:
#from scipy.io import wavfile
import scipy.stats
import numpy as np
import librosa
from tensorflow import keras
import pandas as pd
from pandas import Series

GET DATA FROM GOOGLE DRIVE

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls "/content/gdrive/My Drive"

In [0]:
s_dir = "./gdrive/My Drive/music_speech_classification_dataset/sdata_.csv"
m_dir = "./gdrive/My Drive/music_speech_classification_dataset/mdata_.csv"
test_dir = "./gdrive/My Drive/music_speech_classification_dataset/test_data_.csv"
eval_dir_s= "./gdrive/My Drive/music_speech_classification_dataset/eval_sdata_.csv"
eval_dir_m = "./gdrive/My Drive/music_speech_classification_dataset/eval_mdata_.csv"

In [0]:
s_rawdata = pd.read_csv(s_dir)
m_rawdata = pd.read_csv(m_dir)
test_rawdata = pd.read_csv(test_dir)
evs_rawdata = pd.read_csv(eval_dir_s)
evm_rawdata = pd.read_csv(eval_dir_m)

BUILDING THE DATASET

In [5]:
#samprle rate
sr= 22050

#check data_samples
print ('s_rawdata length:', len(s_rawdata))
print ('s_rawdata: (sr)%d * 5min9s = '%sr, ((5*60)+9)*sr)

print ('m_rawdata length:', len(m_rawdata))
print ('m_rawdata: (sr)%d * 4m59s = '%sr, ((5*60)-1)*sr)

print ('test_rawdata length:', len(test_rawdata))
print ('test_rawdata: (sr)%d * 2min = '%sr, (2*60)*sr)

s_rawdata length: 6822047
s_rawdata: (sr)22050 * 5min9s =  6813450
m_rawdata length: 6608858
m_rawdata: (sr)22050 * 4m59s =  6592950
test_rawdata length: 2651595
test_rawdata: (sr)22050 * 2min =  2646000


In [0]:
# convert from dataframe to ndarray
sdata = s_rawdata.to_numpy()
mdata = m_rawdata.to_numpy()
test_data = test_rawdata.to_numpy()
evs = evs_rawdata.to_numpy()
evm = evm_rawdata.to_numpy()

In [111]:
#columns of raw audio
print (sdata.shape)
print (mdata.shape)
print (test_data.shape)
print (evs.shape)
print (evm.shape)

(6822047,)
(6608858,)
(2651595,)
(1993272, 1)
(2012574, 1)


In [112]:
#type
print (type(sdata))
print (type(mdata))
print (type(test_data))
print (type(evs))
print (type(evm))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


Extract Features from raw audio

In [0]:
# this step is taken in order to provide 1D input into libosa.feature.mfcc - (X,1)table_takes it as 2D
mdata =  np.reshape(mdata, len(mdata))
sdata =  np.reshape(sdata, len(sdata))
test_data = np.reshape(test_data, len (test_data))
evs_data = np.reshape(evs, len(evs))
evm_data = np.reshape(evm, len(evm))

#print (evs_data.shape)
#print (evm_data.shape)

Dataset-1 --> MFCCs(20 first)

In [9]:
#Window size : 1024 due to simplicity --> should me far more less

#training mfccs
speech_mfccs = librosa.feature.mfcc(y=sdata, sr = sr, n_mfcc=20, hop_length=1024)
music_mfccs = librosa.feature.mfcc(y=mdata, sr = sr, n_mfcc=20, hop_length=1024)
print ("speech mfccs shape: ", speech_mfccs.shape)
print ("music mffcs shape:", music_mfccs.shape)

#test mfccs
test_mfccs = librosa.feature.mfcc(y=test_data, sr = sr, n_mfcc=20, hop_length=1024)
print ("test mffcs shape:", test_mfccs.shape)

#evaluation mfccs
evs_mfccs = librosa.feature.mfcc(y= evs_data, sr =sr, n_mfcc=20, hop_length= 1024)
evm_mfccs = librosa.feature.mfcc(y= evm_data, sr =sr, n_mfcc=20, hop_length= 1024)
print ("evs mffcs shape:", evs_mfccs.shape)
print ("evm mffcs shape:", evm_mfccs.shape)


speech mfccs shape:  (20, 6663)
music mffcs shape: (20, 6454)
test mffcs shape: (20, 2590)
evs mffcs shape: (20, 1947)
evm mffcs shape: (20, 1966)


In [10]:
#transpose tables --> rows-samples, columns mfccs || CARE TO DO IT ONLY ONCE!
speech_mfccs=speech_mfccs.transpose()
music_mfccs=music_mfccs.transpose()
test_mfccs=test_mfccs.transpose()
evs_mfccs=evs_mfccs.transpose()
evm_mfccs=evm_mfccs.transpose()
print(speech_mfccs.shape)
print (music_mfccs.shape)
print(test_mfccs.shape)
print(evs_mfccs.shape)
print(evm_mfccs.shape)

(6663, 20)
(6454, 20)
(2590, 20)
(1947, 20)
(1966, 20)


In [11]:
#DEFINE LENGTHS AS VARS 
mfcc_len=len(music_mfccs.transpose())# for each of the 20 col-mfccs
samples_length_music = len(music_mfccs)
samples_length_speech = len(speech_mfccs)
ev_len_s = len(evs_mfccs)
ev_len_m = len(evm_mfccs)

print (mfcc_len)
print (samples_length_music)
print (samples_length_speech)
print (ev_len_s)
print (ev_len_m)

20
6454
6663
1947
1966


In [12]:
#prepare tables for enter statistics
#training
mean_music = np.empty(mfcc_len)
std_music = np.empty(mfcc_len)
mean_speech =  np.empty(mfcc_len)#
std_speech = np.empty(mfcc_len)#

#evaluation
e_means = np.empty(mfcc_len)
e_stds = np.empty(mfcc_len)
e_meanm = np.empty(mfcc_len)
e_stdm =np.empty(mfcc_len)

for col in range(mfcc_len):
  #TRAIN_DATA
  #MUSIC STATS
  mean_music[col] =np.mean(music_mfccs[:,col],axis= 0)#για κάθε στήλη done
  #print("mean:" , meanm)
  std_music[col] = np.std(music_mfccs[:,col],axis= 0) #gia κάθε στήλη done
  #print ("std:" , stdm)
  #SPEECH STATS
  mean_speech[col] =np.mean(speech_mfccs[:,col],axis= 0)#mean for each mfcc column
  std_speech[col] = np.std(speech_mfccs[:,col],axis= 0) #std for each mfcc

  #EVALUATION_DATA
  #MUSIC STATS
  e_meanm[col] = np.mean(evm_mfccs[:,col],axis= 0)#mean για κάθε στήλη done
  e_stdm[col] = np.std(evm_mfccs[:,col],axis= 0)
  #SPEECH_STATS
  e_means[col] =np.mean(evs_mfccs[:,col],axis= 0)#mean για κάθε στήλη done
  e_stds[col] = np.std(evs_mfccs[:,col],axis= 0)
  
  #TESTING_DATA_
  # ...............................

print ("music_mfccs shape:", music_mfccs.shape)
print ("music_mfccs type:", type(music_mfccs))


music_mfccs shape: (6454, 20)
music_mfccs type: <class 'numpy.ndarray'>


In [0]:
#initialize tables for normalized features to be stored
#TRAINING_DATA
normalized_music_mfccs = np.empty((samples_length_music,mfcc_len), dtype=float)
normalized_speech_mfccs = np.empty((samples_length_speech,mfcc_len), dtype=float)
#MUSIC
for mfcc_col in range(mfcc_len):#0->19
  for sample_row in range(samples_length_music):#0->sample_lenght of music mfccs table
    normalized_music_mfccs[sample_row][mfcc_col]=(music_mfccs[sample_row,mfcc_col]-mean_music[mfcc_col])/std_music[mfcc_col]
#SPEECH
for mfcc_col in range(mfcc_len):#0->19
  for sample_row in range(samples_length_speech):#0->sample_lenght of speech mfccs table
    normalized_speech_mfccs[sample_row][mfcc_col]=(speech_mfccs[sample_row,mfcc_col]-mean_speech[mfcc_col])/std_speech[mfcc_col]

#print ("normalized_music shape:",normalized_music_mfccs.shape)
#print(".......... \n \n")
#print (normalized_music_mfccs)

In [0]:
#EVALUATION_DATA
e_norm_m = np.empty((ev_len_m,mfcc_len), dtype=float)
e_norm_s = np.empty((ev_len_s,mfcc_len), dtype=float)
print (e_norm_m.shape)
print (e_norm_s.shape)

#ev_MUSIC
for mfcc_col in range(mfcc_len):#0->19
  for sample_row in range(ev_len_m):#0->sample_lenght of evaluation _music mfccs table
    e_norm_m[sample_row][mfcc_col]=(evm_mfccs[sample_row,mfcc_col]-e_meanm[mfcc_col])/e_stdm[mfcc_col]

#ev_SPEECH
for mfcc_col in range(mfcc_len):
  for sample_row in range(ev_len_s):
    e_norm_s[sample_row][mfcc_col]=(evs_mfccs[sample_row,mfcc_col]-e_means[mfcc_col])/e_stds[mfcc_col]


In [0]:
'''
#SHOULD BE REMOVEDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDDD
#normalization-->distance from highest value rather than distance for mean value
print (music_mfccs.shape)
normalized_mdata= librosa.util.normalize(m, norm = np.inf, axis = 0)#
print (normalized_mdata[0:5,1])
''

In [15]:
#add_labels before join data
slabel = np.empty(samples_length_speech,dtype=int)
mlabel = np.empty(samples_length_music,dtype=int)
ev_len_s = len(evs_mfccs) 

es_label = np.empty(ev_len_s,dtype=object)
em_label = np.empty(ev_len_m,dtype=object)

for i in range (samples_length_speech):
  slabel[i]=1  #for 'speech'

for i in range (samples_length_music):
  mlabel[i]=0 #for 'music'
#print (slabel.shape)
#print (mlabel.shape)
#print (mlabel[0:10])
#.........................................
for i in range (ev_len_m):
  em_label[i]='music'

for i in range (ev_len_s):
  es_label[i]='speech'

labeled_sdata= np.c_[normalized_speech_mfccs,slabel]
labeled_mdata= np.c_[normalized_music_mfccs,mlabel]
#print (labeled_mdata.shape)
#print (labeled_sdata.shape)

ev_labeleds = np.c_[e_norm_s,es_label]
ev_labeledm = np.c_[e_norm_m,em_label]
#print (ev_labeleds[ev_len_s-1])
#print (ev_labeledm[ev_len_m-1])
print (ev_labeleds.shape)

(1947, 21)


In [84]:
#join_TRAININGdata
labeled_data= np.r_[labeled_sdata,labeled_mdata]
#print (labeled_data.shape)
print (labeled_data[6663,:])

#join_EVALUATION_data
evaluation_labeled_data= np.r_[ev_labeleds,ev_labeledm]
print (evaluation_labeled_data.shape)
print (evaluation_labeled_data[:,20])#last column

[-0.23121111  1.5267318   0.57473059  0.36540785 -0.96146981  1.27290413
  0.39883748  0.49579265  0.55643261  0.57715026  2.18849897  2.04645403
  1.42426094 -0.06205785 -0.81587401 -1.53251648 -1.71490368 -1.12503171
  0.68593534  0.85678288  0.        ]
(3913, 21)
['speech' 'speech' 'speech' ... 'music' 'music' 'music']


In [0]:
"""
#suffle data -  should be done only if we had more than 1 files for training
np.random.shuffle(labeled_data)
print ("labeled data:\n",labeled_data[6663,:])

#check suffle
'''
stored random value before suffling
-0.23121110910659604 1.5267318017759424 0.5747305939969028
 0.3654078482502529 -0.961469807074541 1.2729041339190754
 0.39883747511502204 0.4957926548305352 0.5564326117824315
 0.5771502580162926 2.1884989732953675 2.0464540307670216
 1.4242609395642258 -0.06205784889292296 -0.8158740115002427
 -1.532516476392707 -1.7149036762166014 -1.125031714838199
 0.6859353386365241 0.8567828805231822 'music'
'''
result = np.where(labeled_data == -0.23121110910659604)
print (labeled_data[result[0],:])
"""

In [0]:
#EXPORT_Training_DATA
#print("Labeled_Data CHECK 0st row:",labeled_data[0][:])

#Deedn t need to convert directly to columns,rows just had the job done!
#tag = np.empty(shape=(1,21), dtype=object) 

#Add names as the 1st row before exporting
tag = np.empty(shape=(21), dtype=object)
for col in range (len(tag.transpose())):
  if col<(len(tag.transpose())-1):
    tag[col] = ("mfcc"+str(col+1))
  else:
    tag[20]="Labels"

#print (tag.shape)
#print (labeled_data.shape)
train_data = pd.DataFrame(np.array(labeled_data),columns=tag)

#print("Labels CHECK:", train_data.loc[0])
#print("train_data-dataframe: CHECK 1st row:", train_data.loc[1])

train_data.to_csv('Training_set.csv')
#train_data.to_excel("Training_set_check.xlsx")

In [0]:
#EXPORT_Evaluation_DATA
#print("Labeled_Data CHECK 0st row:",evaluation_labeled_data[0][:])

#print (evaluation_labeled_data.shape)
evaluation_data = pd.DataFrame(np.array(evaluation_labeled_data),columns=tag)

#print("Labels CHECK:", evaluation_data.loc[0])

evaluation_data.to_csv('Evaluation_set.csv')
#evaluation_data.to_excel("Evaluation_set_check.xlsx")